<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2021/blob/main/AdiabaticFlash_Broyden.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!git clone --depth 1 https://github.com/profteachkids/chetools.git &> /dev/null
import sys
sys.path.insert(1, "/content/chetools") #Path to CHE module imports

In [10]:
import jax
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True)
import tools.che as che
R=8.314 # J/(mol K)
eps=1e-12

In [11]:
p = che.Props(['Ethanol','Isopropanol', 'Water'])

In [12]:
Ftot=10 # Total Feed moles
Fz = jnp.array([1/3, 1/3, 1/3]) # Equimolar feed composition
FT = 450 # Feed temperature
flashP= 101325 # Flash drum pressure

Vy = Fz # Guess vapor/liquid composition equal to feed
Lx = Fz # Comp - constrains mole fractions to behave like mole fractions!
flashT = jnp.array([360.])  # Guess and bounds for flash temperature
Vtot = jnp.array([Ftot/2])  # Guess half of feed in vapor
Ltot = jnp.array([Ftot/2])

x_guess = jnp.concatenate([Vy, Lx, flashT, Vtot, Ltot])

In [13]:
def model(x):
  Vy = x[0:3]
  Lx = x[3:6]
  flashT = x[6]
  Vtot = x[7]
  Ltot = x[8]

  sum_frac = jnp.array([jnp.sum(Vy)-1., jnp.sum(Lx)-1.])

  V = Vy * Vtot
  L = Lx * Ltot
  F = Fz * Ftot
  mass_balance = F-V-L

  FH = p.Hl(nL=F, T=FT)
  VH = p.Hv(nV=V, T=flashT)
  LH = p.Hl(nL=L, T=flashT)
  energy_balance = jnp.array([FH - VH - LH])

  fugL = Lx  * p.NRTL_gamma(Lx,flashT)* p.Pvap(flashT)
  fugV = Vy*flashP
  VLE = fugL - fugV
  return jnp.concatenate([sum_frac, mass_balance, energy_balance, VLE])

In [14]:
model(x_guess)

DeviceArray([     0.        ,      0.        ,      0.        ,
                  0.        ,      0.        , -48834.08861275,
              17774.2642706 ,  11514.40996588,   3103.78604499],            dtype=float64)

In [15]:
def broyden3(func, x, J=None, max_iter=100, tol=1e-6, verbose=0, xmax=jnp.inf, xmin=-jnp.inf):
  Jf = jax.jacobian(func) if J is None else J
  J = Jf(x)
  Jinv = jnp.linalg.inv(J)
  f = func(x)

  for i in range(max_iter):
    print(f"\nIter: {i}")
    dx = - Jinv @ f

    alpha = jnp.min(jnp.concatenate([jnp.abs((xmax-x)/dx), jnp.abs((xmin-x)/dx), jnp.array([1.])]))

    while alpha > 0.01:
      dx_try = alpha*dx
      xp = x + dx_try
      fp = func(xp)
      dnorm = jnp.linalg.norm(fp)-jnp.linalg.norm(f)
      if verbose>1:
        print(f"Alpha {alpha}   dnorm {dnorm}  dx_try {dx_try}   f {f}    fp {fp}")
      if dnorm > 0:
        alpha *= 0.5
      else:
        break
    if alpha <= 0.01:
      if verbose>1:
        print("reevaluate J")
      Jinv = jnp.linalg.inv(Jf(x))
      continue

    dx=dx_try
    f= fp
    x= xp
    if verbose>0:
      print(i, x, f)
      print()
    if jnp.linalg.norm(fp) < tol:
      break

    u = jnp.expand_dims(fp,1)
    v = jnp.expand_dims(dx,1)/jnp.linalg.norm(dx)**2
    Jinv = Jinv - Jinv @ u @ v.T @ Jinv / (1 + v.T @ Jinv @ u)  #Sherman-Morrison
  return x, f

In [16]:
broyden3(model, x_guess, verbose=2, max_iter=100,
         xmin=jnp.array([0., 0., 0., 0., 0., 0., 300., 0., 0.]),
         xmax=jnp.array([1., 1., 1., 1., 1., 1., FT, Ftot, Ftot]))


Iter: 0
Alpha 1.0   dnorm -51255.62716881574  dx_try [ 0.02472278  0.00966017 -0.03438295 -0.02472278 -0.00966017  0.03438295
 -6.25663839 -1.05743059  1.05743059]   f [     0.              0.              0.              0.
      0.         -48834.08861275  17774.2642706   11514.40996588
   3103.78604499]    fp [-1.11022302e-16  0.00000000e+00  5.22852386e-02  2.04299289e-02
 -7.27151675e-02  3.42515698e+02  1.73998792e+03  1.01485074e+03
  2.87058476e+02]
0 [3.58056109e-01 3.42993508e-01 2.98950383e-01 3.08610558e-01
 3.23673158e-01 3.67716284e-01 3.53743362e+02 3.94256941e+00
 6.05743059e+00] [-1.11022302e-16  0.00000000e+00  5.22852386e-02  2.04299289e-02
 -7.27151675e-02  3.42515698e+02  1.73998792e+03  1.01485074e+03
  2.87058476e+02]


Iter: 1
Alpha 1.0   dnorm -1798.6834991459546  dx_try [ 0.0088231   0.00112228 -0.00994538  0.00270865  0.00338363 -0.00609228
 -0.68971806  0.02944596 -0.02944596]   f [-1.11022302e-16  0.00000000e+00  5.22852386e-02  2.04299289e-02
 -7.27151675

(DeviceArray([3.66566592e-01, 3.43235669e-01, 2.90197739e-01,
              3.11417749e-01, 3.26803264e-01, 3.61778987e-01,
              3.52966203e+02, 3.97389745e+00, 6.02610255e+00],            dtype=float64),
 DeviceArray([-1.11022302e-16,  0.00000000e+00,  2.53130850e-13,
              -2.31592523e-13, -2.17603713e-14,  8.38190317e-09,
               4.70754458e-09, -1.26601662e-09, -1.77169568e-09],            dtype=float64))